## ch 5_11. 데이터 전처리하기

크롤링을 통해 수집한 뉴스 기사 데이터에서 필요없는 부분을 먼저 제거하겠습니다. 텍스트 데이터를 전처리할 때는 정규 표현식을 활용하면 좋습니다.

## 불필요한 텍스트 제거
- 기사 시작 부분에 언론사나 기자 이름을 언급하는 부분
- 기사 마지막 부분에 사진 출처를 언급하는 부분

### 전처리 하기 전 텍스트 확인

In [8]:
import csv
from tqdm import tqdm

data = []
filtered_cnt = 0 
with open("./data/baseball_news.csv") as fr:
    reader = csv.reader(fr)
    next(reader)
    for row in tqdm(reader):
        url, date_str, title, content = row
        
        # 제목이나 본문이 비어있는 기사는 제외
        if not title or not content:
            filtered_cnt += 1
            continue
        data.append(row)
print("total data: ", len(data), "filtered:", filtered_cnt)

102919it [00:01, 79891.17it/s]

total data:  102520 filtered: 399


In [14]:
data[:3]

[['https://sports.news.naver.com/news?oid=117&aid=0003566396',
  '20220101',
  'KBO 역대급 외인타자, 끝내 10개 구단 외면 받고 사라지나',
  '[마이데일리 = 윤욱재 기자] KBO 리그 10개 구단의 외국인선수 구성도 막바지를 향하고 있다. 이미 대부분 구단들은 외국인타자 계약을 완료했다. \'안타 기계\' 호세 미구엘 페르난데스와 재계약을 추진하고 있는 두산을 제외하면 모든 구단이 2022년에 그라운드를 누빌 외국인타자를 확정한 상태다.키움은 메이저리그 통산 132홈런을 터뜨린 야시엘 푸이그를 전격 영입하면서 화제를 뿌렸다. KT는 헨리 라모스, SSG는 케빈 크론, KIA는 소크라테스 브리토, LG는 리오 루이즈, 롯데는 D.J. 피터스, 한화는 마이크 터크먼, NC는 닉 마티니를 새로 수혈했다. 삼성은 호세 피렐라와 재계약을 맺었다.결국 KBO 리그에서 \'역대급 외국인타자\'로 활약했던 그는 10개 구단의 외면을 받고 말 것인가. 바로 에릭 테임즈의 이야기다.테임즈는 2014년 NC에 입단해 처음으로 한국 무대를 밟았다. 타율 .343 37홈런 121타점 11도루를 기록하며 NC의 창단 첫 포스트시즌 진출을 이끌었던 테임즈는 2015년에는 타율 .381 47홈런 140타점 40도루라는 믿기지 않는 성적으로 한국 무대를 완전 정복했다. 그가 달성한 40홈런-40도루 클럽은 KBO 리그 역대 1호 기록이었다. KBO 리그 3년차를 맞은 2016년에도 타율 .321 40홈런 121타점 13도루로 기복이 없었다.한국에서 파워히터로 거듭난 테임즈는 메이저리그 구단들의 관심을 받았고 2017년 밀워키에서 뛰면서 빅리그 복귀의 꿈을 이뤘다. 그해 타율 .247 31홈런 63타점으로 그의 파워가 메이저리그에서도 통할 수 있다는 것을 보여줬다.테임즈는 2018년 타율 .219 16홈런 37타점으로 주춤했고 2019년 타율 .247 25홈런 61타점을 기록한 뒤 2020시즌을 앞두고 워싱턴으로 이적했

In [81]:
example_content = data[0][3]

In [82]:
prefix_pattern_one = re.compile(r"^\[.*?\]\s*")

In [84]:
prefix_pattern_one.match(example_content)

<re.Match object; span=(0, 17), match='[마이데일리 = 윤욱재 기자] '>

## 필터링에 사용할 정규 표현식 작성 요령

1. 예시 데이터를 가지고 정규표현식을 작성한 뒤, 결과를 확인한다.
2. 여러 데이터 셋에 대해서도 정규표현식이 의도한 대로 동작하는지 확인한다.

### 전체 데이터 셋에서 100개를 샘플링해서 정규표현식 패턴에 일치하는 텍스트만 출력

In [85]:
import random

random.seed(1234)

def check_pattern_match(data, pattern, num_samples=200):
    samples = random.choices(data, k=num_samples)
    sample_contents = [sample[-1] for sample in samples]
    for sample_content in sample_contents:
        match = re.match(pattern, sample_content)
        if match:
            print(match)

### 기사 본문 맨 앞에 꺽쇠와 함께 기자 이름, 언론사 이름이 등장하는 경우 필터링
- 예시) [마이데일리 = 장윤호 기자]

In [86]:
prefix_pattern_one = re.compile(r"^\[.*?\]\s*")
prefix_pattern_two = re.compile(r"^\(.*?\)\s*")

In [87]:
check_pattern_match(data, prefix_pattern_two)

<re.Match object; span=(0, 9), match='(서울=뉴스1) '>
<re.Match object; span=(0, 10), match='(서울=연합뉴스) '>
<re.Match object; span=(0, 20), match='(엑스포츠뉴스 잠실, 박윤서 기자) '>
<re.Match object; span=(0, 16), match='(엑스포츠뉴스 박윤서 기자) '>
<re.Match object; span=(0, 16), match='(MHN스포츠\xa0박연준\xa0기자) '>
<re.Match object; span=(0, 20), match='(엑스포츠뉴스 고척, 김지수 기자) '>
<re.Match object; span=(0, 20), match='(엑스포츠뉴스 수원, 김지수 기자) '>
<re.Match object; span=(0, 20), match='(엑스포츠뉴스 창원, 윤승재 기자) '>
<re.Match object; span=(0, 16), match='(엑스포츠뉴스 윤승재 기자) '>
<re.Match object; span=(0, 16), match='(엑스포츠뉴스 김지수 기자) '>
<re.Match object; span=(0, 10), match='(서울=연합뉴스) '>
<re.Match object; span=(0, 16), match='(MHN스포츠 박연준 기자) '>
<re.Match object; span=(0, 10), match='(서울=연합뉴스) '>
<re.Match object; span=(0, 16), match='(엑스포츠뉴스 김지수 기자)\xa0'>
<re.Match object; span=(0, 10), match='(인천=연합뉴스) '>
<re.Match object; span=(0, 20), match='(엑스포츠뉴스 수원, 김한준 기자) '>
<re.Match object; span=(0, 10), match='(서울=연합뉴스) '>
<re.Match object; span=

### re.sub 함수를 이용해서 불필요한 텍스트 제거
앞서서 검증한 패턴을 가지고 데이터 셋에서 불필요한 부분을 제거

In [88]:
import re

def filter_prefix(data):
    prefix_pattern_one = re.compile(r"^\[.*?\]\s*")
    prefix_pattern_two = re.compile(r"^\(.*?\)\s*")

    filtered_data = []
    for url, date_str, title, content in tqdm(data):
        # 꺽쇠, 괄호 안에 담긴 언론사, 기자명 제거
        
        content = re.sub(prefix_pattern_one, "", content)
        content = re.sub(prefix_pattern_two, "", content)
        if not content:
            continue
        filtered_data.append((url, date_str, title, content))
    return filtered_data

In [89]:
prefix_filtered_data = filter_prefix(data)

100%|████████████████████████████████████████| 102520/102520 [00:01<00:00, 61443.90it/s]


In [91]:
with open("./data/baseball_preprocessed.csv", "w") as fw:
    writer = csv.writer(fw)
    for row in tqdm(prefix_filtered_data):
        writer.writerow(row)

100%|████████████████████████████████████████| 102481/102481 [00:02<00:00, 46731.11it/s]
